In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git accelerate>=0.16.0 torchvision transformers>=4.25.1 ftfy tensorboard Jinja2 datasets wandb

In [ ]:
!huggingface-cli login

In [ ]:
!wandb login


In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
!git clone https://github.com/katryo/jupyter-train-controlnet-sdxl.git

In [ ]:
from google.colab import drive
import os

root_dir          = "/content"
drive_dir         = os.path.join(root_dir, "drive/MyDrive")
train_controlnet_dir = os.path.join(drive_dir, "train-controlnet")
validation_dir   = os.path.join(train_controlnet_dir, "validation")

def mount_drive(dir):
    if not os.path.exists(drive_dir):
        drive.mount(os.path.dirname(drive_dir))
    output_dir  = os.path.join(train_controlnet_dir, "output")

    return output_dir


output_dir      = mount_drive(drive_dir)

for dir in [train_controlnet_dir, output_dir, validation_dir]:
    os.makedirs(dir, exist_ok=True)

validation_image_path_a = os.path.join(validation_dir, "validation-depth-a.png")
validation_image_path_b = os.path.join(validation_dir, "validation-depth-b.png")

In [ ]:
!accelerate launch ./jupyter-train-controlnet-sdxl/train_controlnet_sdxl.py \
 --pretrained_model_name_or_path="katryo/mkcface-2-diffusers" \
# stabilityai/stable-diffusion-xl-base-1.0 \
 --controlnet_model_name_or_path="diffusers/controlnet-depth-sdxl-1.0" \
 --hub_model_id="controlnet-mkcface-depth-2" \
 --output_dir=$output_dir \
 --dataset_name=katryo/mkcface-depth-1 \
 --mixed_precision="fp16" \
 --resolution=1024 \
#  --resume_from_checkpoint="checkpoint-12000" \
 --image_column="image" \
 --conditioning_image_column="condition_image" \
 --caption_column="image_caption" \
 --learning_rate=1e-5 \
 --max_train_steps=12000 \
 --checkpointing_steps=4000 \
 --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix"\
 --tracker_project_name "controlnet-mkcface-depth-2" \
 --validation_image $validation_image_path_a $validation_image_path_b \
 --validation_prompt "angry, 1girl, solo, long hair, looking at viewer, bangs, brown hair, brown eyes, lips, portrait, close-up" "frown, 1girl, solo, looking at viewer, bangs, blue eyes, blonde hair, lips, portrait, close-up" \
 --validation_steps=2000 \
 --train_batch_size=1 \
 --gradient_accumulation_steps=4 \
 --report_to="wandb" \
 --seed=42 \
 --push_to_hub

In [ ]:
!accelerate launch ./jupyter-train-controlnet-sdxl/train_controlnet_sdxl.py \
 --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
 --hub_model_id="controlnet-facesynthetics-spiga-sdxl-10000" \
 --output_dir=$output_dir \
 --dataset_name=multimodalart/facesyntheticsspigacaptioned \
 --mixed_precision="fp16" \
 --resolution=512 \
 --image_column="image" \
 --conditioning_image_column="spiga_seg" \
 --caption_column="image_caption" \
 --learning_rate=1e-5 \
 --resume_from_checkpoint=checkpoint-10000 \
 --max_train_steps=10000 \
 --checkpointing_steps=5000 \
 --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix"\
 --tracker_project_name "controlnet-spiga-sdxl" \
 --validation_image $validation_image_path_a $validation_image_path_b \
 --validation_prompt "a close-up of a man" "a close-up of a woman" \
 --validation_steps=5000 \
 --train_batch_size=4 \
 --gradient_accumulation_steps=4 \
 --report_to="wandb" \
 --seed=42 \
 --push_to_hub